In [20]:
import pandas as pd
import numpy as np
import re
from mechanize import Browser

Root = "C:/Users/JJvRijn/Documents/Python_Scripts"

In [64]:
DADP = pd.read_json(
    Root + "/Programing/Data/AMP_focussed_data/DADP_data.json", orient='index')

In [106]:
DADP[0]['children'][2]['children'][0]['children'][0]['name']

'SP_A0SN42'

In [107]:
seq = []
ID = []
for j in range(len(DADP[0]['children'])-1):
    seq.append(DADP[0]['children'][j+1]['children'][6]['children'][0]['name'])
    ID.append(DADP[0]['children'][j+1]['children'][0]['children'][0]['name'])

In [185]:
DADP_data = pd.DataFrame([ID, seq], index = ['ID', 'seq'])
DADP_data = DADP_data.transpose()

In [120]:
All_data = pd.read_csv(Root + "/" + "Programing/Data/AMP_focussed_data/complete_data.csv")
data_complete = list(All_data.seq) + seq
new_peptides = list(set(data_complete) - set(All_data.seq))
len(new_peptides)

412

412/921 seq are new, however a part of those might not have data. Still will scrape the website

In [193]:
browser = Browser()
browser.open("http://split4.pmfst.hr/dadp/?a=kartica&id=SP_P31107")
response = browser.reload()
content = response.read()
table = content.split(b"</table>")[1]
results = table.split(b"</td>")
toxic_type = results[5].split(b"<td>",1)[1]
toxic_amount = re.findall(r"[-+]?\d*\.\d+|\d+", str(results[-4]))[1]
print (toxic_type, toxic_amount)

b'HC50&nbsp;(&mu;M)' 250.00


In [197]:
len(re.findall(r"[-+]?\d*\.\d+|\d+", str(results[-4]))) >= 2

True

In [198]:
DADP_data = DADP_data[DADP_data['seq'].isin(new_peptides)]
DADP_data = DADP_data.reset_index(drop=True)

In [199]:
toxic = []
toxic_type =[]
for r in range(len(DADP_data)):
    URL = "http://split4.pmfst.hr/dadp/?a=kartica&id=" + str(DADP_data['ID'][r])
    browser = Browser()
    browser.open(URL)
    response = browser.reload()
    content = response.read()
    table = content.split(b"</table>")[1]
    results = table.split(b"</td>")
    toxic_type.append(results[5].split(b"<td>",1)[1])
    if len(re.findall(r"[-+]?\d*\.\d+|\d+", str(results[-4]))) >= 2:
        toxic.append(re.findall(r"[-+]?\d*\.\d+|\d+", str(results[-4]))[1])
    else:
        toxic.append("Nan")
DADP_data['CONCENTRATION_µM'] = toxic
DADP_data = DADP_data[DADP_data["CONCENTRATION_µM"] != "Nan"]
DADP_data = DADP_data.reset_index(drop=True)

In [212]:
DADP_data['toxic'] = "~"
for v in range(len(DADP_data)):
    if float(DADP_data["CONCENTRATION_µM"][v]) <= 300:
        DADP_data.loc[v, "toxic"] = 1
DADP_data = DADP_data[DADP_data["toxic"]!= "~"]
DADP_data = DADP_data.reset_index(drop=True)

,ID,seq,CONCENTRATION_µM,toxic
0,SP_A0AAS3,ILPILSLIGGLLGK,180,1
1,SP_Q06B80,ILPILSLIGGLLGK,180,1
2,SP_B8QSS4,FFGTALKIAANVLPTAICKILKKC,0.58,1
3,SP_A6MAW5,GLFTLIKGAAKLIGKTVAKEAGKTGLELMACKITNQC,100,1
4,SP_A6MAX7,GLFTLIKGAAKLIGKTVAKEAGKTGLELMACKITNQC,100,1
5,SP_A6MAW7,GLFTLIKGAAKLIGKTVAKEAGKTGLELMACKITNQC,100,1
6,SP_A6MAY0,GLFTLIKGAAKLIGKTVAKEAGKTGLELMACKITNQC,100,1
7,SP_A6MAW9,GLFTLIKGAAKLIGKTVAKEAGKTGLELMACKITNQC,100,1
8,SP_A6MAX4,GLFTLIKGAAKLIGKTVAKEAGKTGLELMACKITNQC,100,1
9,SP_A6MAX8,GLFTLIKGAAKLIGKTVAKEAGKTGLELMACKITNQC,100,1


In [215]:
peptides = DADP_data["seq"].unique()
final_data = pd.DataFrame(peptides, columns=['seq'])
for a in range(len(DADP_data)):
    for n in range(len(peptides)):
        toxic = []
        toxic_count = 0
        if DADP_data["seq"][a] == peptides[n]:
            toxic.append(float(DADP_data["toxic"][a]))
            toxic_count = sum(toxic)/len(toxic)
            if toxic_count == len(toxic)/2:
                final_data.loc[n, "toxic"] = "~"
            if toxic_count <= len(toxic)/2:
                final_data.loc[n, "toxic"] = 0
            if toxic_count >= len(toxic)/2:
                final_data.loc[n, "toxic"] = 1

In [220]:
final_data.to_csv(
    Root + "/Programing/Data/AMP_focussed_data/DADP_data.csv",index = False, header=True)

In [1]:
final_data

NameError: name 'final_data' is not defined